In [1]:
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn import random_projection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import homogeneity_score
from sklearn.mixture import GaussianMixture
from collections import Counter
import pandas as pd
import numpy as np
import random
from pytictoc import TicToc
t = TicToc()

In [2]:
vocabulary = pd.read_csv("vocab.nytimes.csv",header=None,names=['ID','WORD'] )
vocabulary.shape

(102660, 2)

In [3]:
for ite in range(44442,vocabulary.shape[0]):
    aux_word = vocabulary["WORD"][ite]
    vocabulary["WORD"][ite] = aux_word[4:]

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_14392\417027552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vocabulary["WORD"][ite] = aux_word[4:]


In [4]:
#vocabulary = vocabulary[~vocabulary['WORD'].str.contains('_', na=False, regex=False)]

In [5]:
vocabulary.shape

(102660, 2)

In [6]:
corpus = pd.read_csv("docword.nytimes.txt",header=None,names=['ARTICLE','WORD','OBSERVATIONS'],sep=' ' )

In [7]:
random.seed(19830526)
corpus.shape

(69679427, 3)

In [8]:
basePoliticsWords = ['republican','democratic','senate','congress','senator','representative']
basePoliticsWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(basePoliticsWords)].values.tolist()
articlesPolitics = corpus['ARTICLE'][corpus['WORD'].isin(basePoliticsWordsIDs)].values.tolist()
print(len(articlesPolitics))
selectedPolitics = random.sample(articlesPolitics,2000)
basePoliticsWordsIDs

123418


[7886,
 10185,
 32801,
 32842,
 35020,
 35021,
 55369,
 58011,
 88569,
 88587,
 92424,
 92479]

In [9]:
baseSportWords = ['sport','player','score','competition','championship','mvp']
baseSportWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseSportWords)].values.tolist()
articlesSports = corpus['ARTICLE'][corpus['WORD'].isin(baseSportWordsIDs)].values.tolist()
print(len(articlesSports))
selectedSports = random.sample(articlesSports,2000)
baseSportWordsIDs

115066


[6101, 7518, 25453, 29176, 34560, 37194, 53285, 55276, 81367, 91809]

In [10]:
baseNYWords = ['new_york','manhattan','queens','harlem','brooklyn','new_jersey']
baseNYWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseNYWords)].values.tolist()
articlesNY = corpus['ARTICLE'][corpus['WORD'].isin(baseNYWordsIDs)].values.tolist()
print(len(articlesNY))
selectedNY = random.sample(articlesNY,2000)
baseNYWordsIDs

90905


[4723, 17861, 23359, 51396, 66202, 77181, 82350, 82418]

In [11]:
baseClimateWords = ['climate','hurricane','storm','temperature','snow','rain']
baseClimateWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseClimateWords)].values.tolist()
articlesClimate = corpus['ARTICLE'][corpus['WORD'].isin(baseClimateWordsIDs)].values.tolist()
print(len(articlesClimate))
selectedClimate = random.sample(articlesClimate,2000)
baseClimateWordsIDs

31652


[6826, 19013, 31357, 36511, 37873, 39515, 54738, 67855, 87645, 95253]

In [12]:
baseEducationWords = ['university','graduate','student','school','teacher','learning']
baseEducationWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseEducationWords)].values.tolist()
articlesEducation = corpus['ARTICLE'][corpus['WORD'].isin(baseEducationWordsIDs)].values.tolist()
print(len(articlesEducation))
selectedEducation = random.sample(articlesEducation,2000)
baseEducationWordsIDs

141960


[17033, 22130, 34498, 38097, 39341, 41880, 65170, 95365, 96227, 99200]

In [13]:
baseTechWords = ['software','internet','data','cybersecurity','technology','digital']
baseTechWordsIDs = vocabulary['ID'][vocabulary['WORD'].isin(baseTechWords)].values.tolist()
articlesTech = corpus['ARTICLE'][corpus['WORD'].isin(baseTechWordsIDs)].values.tolist()
print(len(articlesTech))
selectedTech = random.sample(articlesTech,2000)
baseTechWordsIDs

90350


[9422, 9602, 10811, 20570, 36639, 39398, 68584]

In [14]:

selectedArticles = selectedTech + selectedPolitics + selectedNY + selectedClimate + selectedEducation + selectedSports
selectedWords = baseTechWordsIDs + basePoliticsWordsIDs + baseNYWordsIDs + baseClimateWordsIDs + baseEducationWordsIDs + baseSportWordsIDs
print(len(selectedArticles))
#selectedArticles = list(set(selectedArticles))
#print(len(selectedArticles))
eliminating=Counter(selectedArticles)
selectedArticlesSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedArticlesSplit.append(ite)
print(len(selectedArticlesSplit))
corpus = corpus[corpus['ARTICLE'].isin(selectedArticles)]
#words_to_keep = vocabulary['ID'].tolist()
#corpus = corpus[corpus['WORD'].isin(words_to_keep)]
corpus.shape
              

12000
11413


(3216090, 3)

In [15]:
corpus = corpus.pivot(index='ARTICLE',columns='WORD',values='OBSERVATIONS')
corpus = corpus.fillna(0)
selectedWords = selectedWords + random.sample(corpus.columns.values.tolist(),10000)    
eliminating=Counter(selectedWords)
selectedWordsSplit = []
for ite in eliminating:
    if eliminating[ite] == 1:
        selectedWordsSplit.append(ite)

In [16]:
print(corpus.shape)
#corpus = corpus.loc[:, (corpus != 0).any(axis=0)]
print(len(selectedWordsSplit))
corpus = corpus[selectedWordsSplit]
print(corpus.shape)

(11702, 80879)
10039
(11702, 10039)


In [17]:

label_real = []
for row in corpus.index:
    if row in selectedPolitics:
        label_real.append(0)
    elif row in selectedTech:
        label_real.append(1)
    elif row in selectedEducation:
        label_real.append(2)
    elif row in selectedNY:
        label_real.append(3)
    elif row in selectedClimate:
        label_real.append(4)
    elif row in selectedSports:
        label_real.append(5)

In [18]:
homogeneity_sol = pd.DataFrame([['Test',0,0,0.000,0.000]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,Time
0,Test,0,0,0.0,0.0


In [19]:
corpus = corpus.fillna(0)
corpus[corpus > 1] = 1
#columnas = list(corpus.columns)
#columnas_words = vocabulary["ID"]=

    

In [20]:
t.tic()
pca = PCA(n_components=100, random_state = 19830526)
pca.fit(corpus)
t.toc('PCA Calibration with 200 components')

PCA Calibration with 200 components 42.127945 seconds.


In [21]:
np.cumsum(pca.explained_variance_ratio_)

array([0.02092258, 0.03647718, 0.04927263, 0.05800725, 0.06648504,
       0.07362777, 0.08049479, 0.08677015, 0.09289918, 0.0988467 ,
       0.10441486, 0.10981384, 0.11517325, 0.12044074, 0.12560554,
       0.13067592, 0.13555163, 0.14026628, 0.14496009, 0.14949293,
       0.1539507 , 0.15830466, 0.16252634, 0.16672376, 0.17076541,
       0.17479284, 0.17872025, 0.18263653, 0.18642002, 0.19009205,
       0.19375883, 0.19733089, 0.20087192, 0.20431493, 0.2077308 ,
       0.21107794, 0.21439909, 0.21768218, 0.22093654, 0.22413086,
       0.22730047, 0.23043201, 0.23353029, 0.23659592, 0.23962298,
       0.24263722, 0.24562125, 0.24856243, 0.25148809, 0.25440066,
       0.25727584, 0.26012923, 0.26295331, 0.26576077, 0.26855491,
       0.2713003 , 0.27402115, 0.27670981, 0.27937382, 0.28201933,
       0.28461518, 0.2872082 , 0.28978526, 0.29235098, 0.29489191,
       0.29742531, 0.29990251, 0.30234589, 0.30476601, 0.30717449,
       0.30957227, 0.31193908, 0.31428174, 0.31660059, 0.31887

In [22]:
sum(pca.explained_variance_ratio_)

0.3702156481176338

In [23]:
t.tic()
corpus_pca = pca.transform(corpus)
t.toc('PCA transform took')

PCA transform took 4.034193 seconds.


In [24]:
labels_pca = []
for ite in range(1,20):
    for iteCom in range(1,50):
        t.tic()
        gm_label_pca = GaussianMixture(n_components=ite, random_state=19830526).fit_predict(corpus_pca[:,0:iteCom])         
        labels_pca.append(gm_label_pca)
        taux = t.tocvalue()
        print("PCA with "+str(iteCom)+" features and "+str(ite)+" clusters took to run " + str(taux))
        new_row = pd.DataFrame([["PCA",iteCom,ite,homogeneity_score(label_real,gm_label_pca),taux]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
        homogeneity_sol = pd.concat([homogeneity_sol,new_row])

PCA with 1 features and 1 clusters took to run 3.008301000000017
PCA with 2 features and 1 clusters took to run 0.1034273000000212
PCA with 3 features and 1 clusters took to run 0.047482899999977235
PCA with 4 features and 1 clusters took to run 0.044804199999987304
PCA with 5 features and 1 clusters took to run 0.13142239999996264
PCA with 6 features and 1 clusters took to run 0.14033009999997148
PCA with 7 features and 1 clusters took to run 0.15876959999997098
PCA with 8 features and 1 clusters took to run 0.14319389999997156
PCA with 9 features and 1 clusters took to run 0.15982239999999592
PCA with 10 features and 1 clusters took to run 0.14383989999998903
PCA with 11 features and 1 clusters took to run 0.2141043000000309
PCA with 12 features and 1 clusters took to run 0.21234650000002375
PCA with 13 features and 1 clusters took to run 0.273255000000006
PCA with 14 features and 1 clusters took to run 0.22404679999999644
PCA with 15 features and 1 clusters took to run 0.24342150000

PCA with 27 features and 3 clusters took to run 2.4300708999999188
PCA with 28 features and 3 clusters took to run 3.579289900000049
PCA with 29 features and 3 clusters took to run 3.531221500000015
PCA with 30 features and 3 clusters took to run 3.3813966000000164
PCA with 31 features and 3 clusters took to run 3.933339499999988
PCA with 32 features and 3 clusters took to run 4.404221000000007
PCA with 33 features and 3 clusters took to run 5.18155969999998
PCA with 34 features and 3 clusters took to run 3.965900599999941
PCA with 35 features and 3 clusters took to run 4.205614300000093
PCA with 36 features and 3 clusters took to run 5.099407000000042
PCA with 37 features and 3 clusters took to run 5.788037900000063
PCA with 38 features and 3 clusters took to run 3.4012974999999415
PCA with 39 features and 3 clusters took to run 4.88760000000002
PCA with 40 features and 3 clusters took to run 4.47253969999997
PCA with 41 features and 3 clusters took to run 3.663464299999987
PCA with 4

PCA with 4 features and 6 clusters took to run 0.6041533000000072
PCA with 5 features and 6 clusters took to run 1.3537802999999258
PCA with 6 features and 6 clusters took to run 1.8718745999999555
PCA with 7 features and 6 clusters took to run 1.2632023000001027
PCA with 8 features and 6 clusters took to run 1.5574286000000939
PCA with 9 features and 6 clusters took to run 2.7905848000000333
PCA with 10 features and 6 clusters took to run 2.6762377000000015
PCA with 11 features and 6 clusters took to run 3.015990299999885
PCA with 12 features and 6 clusters took to run 4.895009599999867
PCA with 13 features and 6 clusters took to run 6.769231499999933
PCA with 14 features and 6 clusters took to run 6.47827570000004
PCA with 15 features and 6 clusters took to run 3.938028000000031
PCA with 16 features and 6 clusters took to run 9.963760999999977
PCA with 17 features and 6 clusters took to run 8.561357900000075
PCA with 18 features and 6 clusters took to run 5.440726300000051
PCA with 1

PCA with 30 features and 8 clusters took to run 27.00082119999979
PCA with 31 features and 8 clusters took to run 29.949141500000223
PCA with 32 features and 8 clusters took to run 23.59415369999988
PCA with 33 features and 8 clusters took to run 20.62093939999977
PCA with 34 features and 8 clusters took to run 23.541073900000356
PCA with 35 features and 8 clusters took to run 30.012275099999897
PCA with 36 features and 8 clusters took to run 21.274213000000145
PCA with 37 features and 8 clusters took to run 15.102529599999798
PCA with 38 features and 8 clusters took to run 21.940600099999756
PCA with 39 features and 8 clusters took to run 24.845971199999894
PCA with 40 features and 8 clusters took to run 17.260985900000378
PCA with 41 features and 8 clusters took to run 20.415574500000275
PCA with 42 features and 8 clusters took to run 16.459876899999927
PCA with 43 features and 8 clusters took to run 9.113875999999891
PCA with 44 features and 8 clusters took to run 10.285545799999909

PCA with 6 features and 11 clusters took to run 8.210269600000174
PCA with 7 features and 11 clusters took to run 10.016872000000149
PCA with 8 features and 11 clusters took to run 10.530908499999896
PCA with 9 features and 11 clusters took to run 10.373557199999595
PCA with 10 features and 11 clusters took to run 11.076457899999696
PCA with 11 features and 11 clusters took to run 7.326850100000229
PCA with 12 features and 11 clusters took to run 8.97463130000051
PCA with 13 features and 11 clusters took to run 7.762138899999627
PCA with 14 features and 11 clusters took to run 8.953914699999586
PCA with 15 features and 11 clusters took to run 4.229608100000405
PCA with 16 features and 11 clusters took to run 4.496943100000863
PCA with 17 features and 11 clusters took to run 3.034178000000793
PCA with 18 features and 11 clusters took to run 7.064347600000474
PCA with 19 features and 11 clusters took to run 9.53744479999932
PCA with 20 features and 11 clusters took to run 15.421380699999

...

PCA with 42 features and 17 clusters took to run 116.77547910000067
PCA with 43 features and 17 clusters took to run 142.49481159999777
PCA with 44 features and 17 clusters took to run 110.70790569999735
PCA with 45 features and 17 clusters took to run 66.4484403000024
PCA with 46 features and 17 clusters took to run 41.40415969999958
PCA with 47 features and 17 clusters took to run 62.60762439999962
PCA with 48 features and 17 clusters took to run 37.900701999999
PCA with 49 features and 17 clusters took to run 159.30460239999957
PCA with 1 features and 18 clusters took to run 0.6317010999991908
PCA with 2 features and 18 clusters took to run 1.6758071999975073
PCA with 3 features and 18 clusters took to run 3.276552499999525
PCA with 4 features and 18 clusters took to run 4.503388399996766
PCA with 5 features and 18 clusters took to run 11.152654200002871
PCA with 6 features and 18 clusters took to run 15.978988800001389
PCA with 7 features and 18 clusters took to run 21.309688299999

D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


PCA with 38 features and 18 clusters took to run 77.12739080000028
PCA with 39 features and 18 clusters took to run 129.31346029999986
PCA with 40 features and 18 clusters took to run 85.96702899999946
PCA with 41 features and 18 clusters took to run 102.89072770000348
PCA with 42 features and 18 clusters took to run 54.7286509000005
PCA with 43 features and 18 clusters took to run 40.003409799999645
PCA with 44 features and 18 clusters took to run 53.79159900000013


D:\ProgramData\Anaconda3\envs\ml\lib\site-packages\sklearn\mixture\_base.py:286: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


PCA with 45 features and 18 clusters took to run 132.47576760000084
PCA with 46 features and 18 clusters took to run 111.30454729999838
PCA with 47 features and 18 clusters took to run 177.87167970000155
PCA with 48 features and 18 clusters took to run 43.182359200000064
PCA with 49 features and 18 clusters took to run 26.52464340000006
PCA with 1 features and 19 clusters took to run 0.18295349999971222
PCA with 2 features and 19 clusters took to run 0.5359522999970068
PCA with 3 features and 19 clusters took to run 0.8824512000028335
PCA with 4 features and 19 clusters took to run 1.2472403000028862
PCA with 5 features and 19 clusters took to run 2.0712280000007013
PCA with 6 features and 19 clusters took to run 2.3200744000023406
PCA with 7 features and 19 clusters took to run 3.1303737000016554
PCA with 8 features and 19 clusters took to run 4.7110186999998405
PCA with 9 features and 19 clusters took to run 4.490822199997638
PCA with 10 features and 19 clusters took to run 7.9225048

In [25]:
homogeneity_sol

,Algorithm,Components,Clusters,Homogeneity,Time
0,Test,0,0,0.000000,0.000000
0,PCA,1,1,0.000000,3.008301
0,PCA,2,1,0.000000,0.103427
0,PCA,3,1,0.000000,0.047483
0,PCA,4,1,0.000000,0.044804
...,...,...,...,...,...
0,PCA,45,19,0.155160,140.806442
0,PCA,46,19,0.149520,120.243885
0,PCA,47,19,0.150105,224.087027
0,PCA,48,19,0.159091,184.442608


In [30]:
labels = []
for ite in range(7,30):
    t.tic()
    gm_label = GaussianMixture(n_components=ite, random_state=19830526).fit_predict(corpus)         
    labels.append(gm_label)
    taux = t.tocvalue()
    print("RAW with "+str(ite)+" clusters took to run " + str(taux))
    new_row = pd.DataFrame([["RAW",0,ite,homogeneity_score(label_real, gm_label),taux]],columns=["Algorithm","Components","Clusters","Homogeneity","Time"])
    homogeneity_sol = pd.concat([homogeneity_sol,new_row])

MemoryError: Unable to allocate 5.26 GiB for an array with shape (7, 10039, 10039) and data type float64

In [ ]:
#ICA = FastICA(n_components=200,max_iter=1000)
#corpus_ica=ICA.fit_transform(corpus)

In [31]:
print(corpus.shape)
print(len(label_real))

(11702, 10039)
11702


In [32]:
df = pd.DataFrame(label_real, index=corpus.index ,columns =['Real'])

In [33]:
homogeneity_sol.to_csv("BagOfWordsClusteringResultsEM.csv")

In [34]:
df_pca = pd.DataFrame(labels_pca, columns=corpus.index).T
df_raw = pd.DataFrame(labels, columns=corpus.index).T
df.to_csv("actualsEM.csv")
df_pca.to_csv("pcaEM.csv")
df_raw.to_csv("rawEM.csv")